In [612]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tslearn
import math
import random
import typing
import copy
from tslearn.utils import to_time_series
from tslearn.metrics import dtw_path,dtw_path_from_metric


In [613]:
raw_points_data_filename =  "_points.json"

with open(raw_points_data_filename, 'r') as f:
    json_data = json.load(f) 

p_data = json_data 
points = []
for d in p_data:
    points.extend(d['points'])

In [614]:
def normalize_wifi_points(points):
    wpoints = []
    for p in points:
        wp = {
            "x":p['x'],
            "y":p['y'],
            "floor":p['floor'],
            "wifiData": p['wifiData'],
            "gpsData": p['gpsData'],
            "timestamp":p['timestamp']
        }
        wpoints.append(wp)
    wifi_points = [w for w in wpoints if w['wifiData']]    
    return wifi_points

def print_json(data):
    print(json.dumps(data,indent=2))


def get_num_interpolation(f1,f2):
    n = math.floor(f1 / f2)
    return n

def pad_vector(vector, length):
    """Pad or truncate vector to a specified length."""
    if len(vector) >= length:
        return vector[:length]  # Truncate if longer than desired length
    else:
        return vector + [0] * (length - len(vector))  # Pad with zeros if shorter

def format_float_to_two_decimals(number):
    formatted_number_str = f"{number:.2f}"  # Format as string
    formatted_number = float(formatted_number_str)  # Convert back to float
    return formatted_number


In [615]:
def get_ap_occurances(points):
    ap_occurances = {}
    for p in points:
        for w in p['wifiData']:
            if not w['BSSID'] in ap_occurances:
                ap_occurances[w['BSSID']] = 0
            ap_occurances[w['BSSID']] += 1
    return ap_occurances
    
def filter_by_networks(point,nets):
    point['wifiData'] = [p for p in point['wifiData'] if p['SSID'] in nets]

def filter_by_rss_threshold(point,signal_strength_threshold):
    point['wifiData'] = [p for p in point['wifiData'] if p['level'] >= signal_strength_threshold]

def filter_by_ap_num_occurances(point,ap_occurances,ap_occurance_threshold):
    point['wifiData'] = [p for p in point['wifiData'] if ap_occurances[p['BSSID']] >= ap_occurance_threshold]
    
def get_rss(point):
    return [p['level'] for p in point['wifiData']]

def remove_empty_reference_points(points):
    return [
        p for p in points 
        if 'wifiData' in p and isinstance(p['wifiData'], list) and len(p['wifiData']) > 0
    ]
    
def filter_dataset_by_bssids(dataset, target_point, at_least_threshold):
    # Extract BSSIDs from the target_point's WiFi data
    target_bssids = {p['BSSID'] for p in target_point['wifiData']}

    filtered_dataset = []
    
    for point, vector in dataset:
        # Extract BSSIDs from the current point's vector
        point_bssids = {p['BSSID'] for p in vector}

        # Count how many BSSIDs of the target point are present in the current point
        common_bssids = target_bssids.intersection(point_bssids)
        
        # Check if the number of common BSSIDs meets the threshold
        if len(common_bssids) >= at_least_threshold:
            filtered_dataset.append((point, vector))

    return filtered_dataset


def filter_by_vector_length(dataset, length_threshold):
    filtered_dataset = []
    
    for point, vector in dataset:
        # Check if the length of the vector meets the threshold
        if len(vector) >= length_threshold:
            filtered_dataset.append((point, vector))

    return filtered_dataset

    

def get_same_aps(point1,point2):
    np1 = point1.copy()
    np2 = point2.copy()
    np1['wifiData'] = []
    np2['wifiData'] = []
    for p1 in point1['wifiData']:
        for p2 in point2['wifiData']:
            if p1['SSID'] == p2['SSID'] and p1['BSSID'] == p2['BSSID']:
                np1['wifiData'].append(p1)
                np2['wifiData'].append(p2)
    return np1,np2



def linear_interpolation(rss1, rss2, n, i):
    return ((n - i) / n) * rss1 + (i / n) * rss2


def wifi_linear_interpolation(point1, point2,num_interpolation):
    rsses = []
    for p1 in point1['wifiData']:
        found = False
        for p2 in point2['wifiData']:
            if p1['SSID'] == p2['SSID'] and p1['BSSID'] == p2['BSSID']:
                rss = {
                    "RSS": linear_interpolation(p1['level'],p2['level'], num_interpolation,len(rsses)),
                    "SSID":p1['SSID'],
                    "BSSID":p1['BSSID'],
                }
                rsses.append(rss)
            found = True
        if not found:
            raise ValueError("invalid points")
    return rsses




def create_map(points):
    network_values = {}
    bssid_values = {}
    network_int = 0
    bssid_int = 0
    for p in points:
        wifi = p['wifiData']
        for w in wifi: 
            ssid = w['SSID']
            bssid = w['BSSID']
            if ssid not in network_values:
                network_values[ssid] = network_int
            if bssid not in network_values:
                bssid_values[bssid] = bssid_int

    return network_values,bssid_values


def create_access_point_index(points):
    """ Create a mapping of unique BSSID to index """
    bssid_to_index = {}
    index = 0
    for point in points:
        for ap in point['wifiData']:
            bssid = ap['BSSID']
            if bssid not in bssid_to_index:
                bssid_to_index[bssid] = index
                index += 1
    return bssid_to_index,index



def create_feature_vectors(points, bssid_to_index):
    """ Create feature vectors with standardized dimensions """
    feature_vectors = []
    for point in points:
        vector = np.zeros(len(bssid_to_index))  # Initialize vector with zeros
        for ap in point['wifiData']:
            bssid = ap['BSSID']
            if bssid in bssid_to_index:
                vector[bssid_to_index[bssid]] = ap['level']
        feature_vectors.append(vector)
    return feature_vectors
    

In [616]:
def normalize_min_max(values, min_value, max_value):
    return [(value - min_value) / (max_value - min_value) for value in values]

In [617]:
def cosine_distance(vector1, vector2):
    dot_product = sum(v1 * v2 for v1, v2 in zip(vector1, vector2))
    norm1 = math.sqrt(sum(v1 ** 2 for v1 in vector1))
    norm2 = math.sqrt(sum(v2 ** 2 for v2 in vector2))
    if norm1 == 0 or norm2 == 0:
        return 1  # Handle zero vector case
    return 1 - dot_product / (norm1 * norm2)


def manhattan_distance(vector1, vector2):
    return sum(abs(v1 - v2) for v1, v2 in zip(vector1, vector2))

def dynamic_time_warping_similarity(vector1, vector2):
    n, m = len(vector1), len(vector2)
    dtw_matrix = [[float('inf')] * (m + 1) for _ in range(n + 1)]
    dtw_matrix[0][0] = 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = abs(vector1[i - 1] - vector2[j - 1])
            dtw_matrix[i][j] = cost + min(dtw_matrix[i - 1][j], dtw_matrix[i][j - 1], dtw_matrix[i - 1][j - 1])

    return 1 / (1 + dtw_matrix[n][m])  # Similarity, so invert the distance

def euclidean_distance(vector1, vector2):
    return math.sqrt(sum((v1 - v2) ** 2 for v1, v2 in zip(vector1, vector2)))


def calculate_distance(v1: typing.List[typing.Union[int, float]], v2: typing.List[typing.Union[int, float]], func: typing.Callable[[typing.List[float], typing.List[float]], float]) -> float:
    if len(v1) != len(v2):
        raise ValueError("Vectors must be of the same length")

    # Ensure elements are float for numeric calculations
    vector1 = list(map(float, v1))
    vector2 = list(map(float, v2))

    # Calculate distance using the specified function
    return func(vector1, vector2)


In [618]:
def create_fingerprint_dataset(points):
    extra = {}
    dataset = []
    min_length = 999
    max_length = -1
    max_rss = -1
    min_rss = 999
    for point in points:
        min_length = min(min_length,len(point['wifiData']))
        max_length = max(max_length,len(point['wifiData']))
        for p in point['wifiData']:
            min_rss = min(min_rss,p['level'])
            max_rss = max(max_rss,p['level'])


    aps,num_aps = create_access_point_index(points)
    extra['max'] = max_length
    extra['min'] = min_length
    extra['aps'] = aps
    extra['num_aps'] = num_aps
    extra['min_rss'] = min_rss
    extra['max_rss'] = max_rss

    for point in points:
        vector = create_fingerprint_vector(point,extra)
        p = point['x'],point['y'],point['floor']
        dataset.append((p,vector))
    return dataset,extra

def filter_dataset(dataset,point):
    at_least_threshold = 1
    num_rsses = len(point['wifiData'])
    filtered_dataset = dataset
    # filtered_dataset = filter_dataset_by_bssids(dataset,point,at_least_threshold)
    return filtered_dataset

def create_fingerprint_feature_vector(point, bssid_to_index,number_bssids):
    """ Create feature vectors with standardized dimensions """
    
    vector = [0 for i in range(number_bssids)]
    for ap in point['wifiData']:
        bssid = ap['BSSID']
        
        try:
            vector[bssid_to_index[bssid]] = ap['level']
        except Exception as e:
            print(e)
            print("BSID index",bssid_to_index[bssid])
            print(vector)
    return vector

    

def create_fingerprint_vector(point,extra):
    max_rss_length = extra['max']

    aps = extra['aps']
    number_aps = extra['num_aps']
    p = (point['x'],point['y'],point['floor'])
    
    vector = create_fingerprint_feature_vector(point,aps,number_aps)
    vector = normalize_min_max(values=vector,min_value=extra['min_rss'],max_value=extra['max_rss'])
    return vector
    # rsses = get_rss(point)
    # normal_vector = pad_vector(rsses,max_rss_length)
    # print(p,"rsses:",rsses,"vec:",normal_vector)
    # return normal_vector

def calculate_weighted_distance(v1, v2):
    matching_aps = sum(1 for a, b in zip(v1, v2) if a != 0 and b != 0)
    total_aps = sum(1 for a, b in zip(v1, v2) if a != 0 or b != 0)
    distance = np.linalg.norm(np.array(v1) - np.array(v2))
    penalty = 1 + (total_aps - matching_aps) / total_aps
    return distance * penalty

def knn(dataset, target_vector, distance_func:typing.Callable,k=1):
    distances = []
    for point,vector in dataset:
        distance = distance_func(vector, target_vector)
        distances.append((point, format_float_to_two_decimals(distance)))

    distances.sort(key=lambda x: x[1])
    nearest_neighbors = distances[0:k]
    return nearest_neighbors

def get_wifi_result(point,points,distance_func,k):
    dataset, extra = create_fingerprint_dataset(points)
    dataset = filter_dataset(dataset,point)
    target_vector = create_fingerprint_vector(point,extra)
    distance_function = lambda v1,v2: calculate_distance(v1,v2,distance_func)
    result = knn(dataset,target_vector,distance_function,k)
    return result

def estimate_position(knn_results):
    x_coords = [point[0] for point, _ in knn_results]
    y_coords = [point[1] for point, _ in knn_results]
    floors = [point[2] for point, _ in knn_results]
    return (format_float_to_two_decimals(sum(x_coords) / len(x_coords)), format_float_to_two_decimals(sum(y_coords) / len(y_coords)), round(sum(floors) / len(floors)))

def estimate_position_by_distance_weight(knn_results):
    x_coords = [point[0] for point, _ in knn_results]
    y_coords = [point[1] for point, _ in knn_results]
    floors = [point[2] for point, _ in knn_results]
    distances = [distance for _, distance in knn_results]
    
    # Convert distances to weights, using inverse distance
    weights = [1 / d if d != 0 else 1 for d in distances]
    
    # Normalize weights
    weight_sum = sum(weights)
    normalized_weights = [w / weight_sum for w in weights]
    
    # Calculate weighted average
    weighted_x = sum(w * x for w, x in zip(normalized_weights, x_coords))
    weighted_y = sum(w * y for w, y in zip(normalized_weights, y_coords))
    weighted_floors = sum(w * f for w, f in zip(normalized_weights, floors))
    
    return (format_float_to_two_decimals(weighted_x), 
            format_float_to_two_decimals(weighted_y), 
            round(weighted_floors))

            
def normalize_scan(point):
    filter_by_networks(point,nets)
    filter_by_rss_threshold(point,signal_strength_threshold)
    return point

def get_expected_result(point,points,distance_func,k):
    distance_function = lambda v1,v2: calculate_distance(v1,v2,distance_func)
    distances = []
    for p in points:
        d = distance_function([point['x'], point['y'], point['floor']], [p['x'], p['y'], p['floor']])
        distances.append(((p['x'],p['y'],p['floor']),format_float_to_two_decimals(d)))
    sorted_distances = sorted(distances,key=lambda x:x[1])
    return sorted_distances[:k]


In [619]:
# f2 = 0.0333
# f1 = 0.0333
# num_interpolation = get_num_interpolation(f1,f2)

signal_strength_threshold = -70
ap_occurance_threshold = 5
nets = ['Afeka-Staff',
'Afeka-Wifi-Open',
'Afeka-Students']


In [620]:


def wifi_workflow(points):
   
    wifi_points = normalize_wifi_points(points)
    ap_occurances = get_ap_occurances(wifi_points)
    for p in wifi_points:
        filter_by_ap_num_occurances(p,ap_occurances,ap_occurance_threshold)
    wifi_points = remove_empty_reference_points(wifi_points)
    dataset_points = copy.deepcopy(wifi_points)

    for p in wifi_points:
        normalize_scan(p)

    wifi_points = remove_empty_reference_points(wifi_points)

    print("with wifi after rss threshold and network filtering, and num ap occurances:",len(wifi_points))  

    p_start = 0
    p_end = 30

    print("testing on points (with the current point):",p_start,"-",p_end)

    # print(dataset_points[0])
    for i,scan_point in enumerate(dataset_points[p_start:p_end]):
        
        x = scan_point['x']
        y = scan_point['y']
        floor = scan_point['floor']
        normalize_point = normalize_scan(scan_point)
        # print(normalize_point)
        if not ((1000 > x > 300) and ( 800 > y > 60 ) and (floor == 0)):
            continue

        if len(normalize_point['wifiData']) == 0:
            print(f"skipping... no wifi data {i}")
            continue

        print(" CURRENT POINT: " ,(x,y,floor))
        best_k = 2
        best_val = 9999
        for k in range(2,100):
            distance_func = euclidean_distance
            wifi_result = get_wifi_result(normalize_point,wifi_points,distance_func=distance_func,k=k)
            expected_result = get_expected_result(normalize_point,wifi_points,distance_func=euclidean_distance,k=k)
            position = estimate_position(wifi_result[1:])
            cur = euclidean_distance([position[0],position[1],position[2]],[x,y,floor])
            if cur < best_val:
                best_k = k
                best_val = cur


        print(f"| expected:{(x,y,floor)} | estimated: {position} | \n| res: {wifi_result} |\n| exp: {expected_result} | \n\n")
        print(f"best k: {best_k}, best val: {best_val}")

        # print(f"| expected:{(x,y,floor)} | | result {wifi_result} |")
        


wifi_workflow(points)

with wifi after rss threshold and network filtering, and num ap occurances: 158
testing on points (with the current point): 0 - 30
 CURRENT POINT:  (426, 90, 0)
| expected:(426, 90, 0) | estimated: (422.66, 346.79, 0) | 
| res: [((426, 90, 0), 0.0), ((140, 100, 0), 0.22), ((176, 35, 0), 0.25), ((511, 35, 0), 0.26), ((210, 100, 0), 0.26), ((261, 35, 0), 0.31), ((35, 100, 0), 1.57), ((346, 35, 0), 1.63), ((111, 35, 0), 1.65), ((36, 0, 0), 1.87), ((36, 35, 0), 1.93), ((576, 35, 0), 2.08), ((426, 35, 0), 2.23), ((35, 0, -1), 2.23), ((0, 35, 0), 2.23), ((35, 5, 0), 2.23), ((656, 5, 0), 2.24), ((626, 35, 0), 2.3), ((751, 35, 0), 2.35), ((425, 15, 0), 2.51), ((35, 316, -1), 2.72), ((625, 250, 0), 2.85), ((685, 275, 0), 2.92), ((242, 338, 0), 2.94), ((362, 338, 0), 2.98), ((192, 378, 0), 2.99), ((327, 283, 0), 3.05), ((327, 338, 0), 3.08), ((35, 1181, -1), 3.09), ((358, 675, -1), 3.14), ((685, 215, 0), 3.16), ((676, 35, 0), 3.18), ((227, 941, -1), 3.22), ((627, 333, 0), 3.24), ((502, 368, 0), 

In [621]:
# w1 = wpoints[48].copy()
# w2 = wpoints[49].copy()
# w3 = wpoints[50].copy()
# w4 = wpoints[51].copy()


# def filter_ap(point):
#     a = len(point['wifiData'])
#     filter_by_networks(point)
#     b = len(point['wifiData'])
#     filter_by_rss_threshold(point,threshold)
#     c = len(point['wifiData'])
#     return (a,b,c)

# def info_wifi(point):
#     info = {}
#     for f in point['wifiData']:

#         if f['SSID'] not in info:
#             info[f['SSID']] = {
#                 'num_rss':0,
#                 'num_aps':0,
#                 'BSSIDs': {}
#             }
        
#         if f["BSSID"] not in info[f['SSID']]['BSSIDs']:
#             info[f['SSID']]['BSSIDs'][f["BSSID"]] = []
#             info[f['SSID']]['num_aps'] += 1


#         info[f['SSID']]['BSSIDs'][f["BSSID"]].append(f['level'])
#         info[f['SSID']]['num_rss'] += 1  

    
#     strings = []
#     for key,value, in info.items():
#         strings.append(str(("net:",key, "num_rss",value['num_rss'], "num_aps", value['num_aps'])))
#     return info,"\n".join(strings)






# print("w1:",w1['x'],w1['y'],w1['floor'], filter_ap(w1))
# print("w2:",w2['x'],w2['y'],w2['floor'], filter_ap(w2))
# print("w3:",w3['x'],w3['y'],w3['floor'], filter_ap(w3))
# print("w4:",w4['x'],w4['y'],w4['floor'], filter_ap(w4))

# # print("w1:",info_wifi(w1)[1])
# # print("w2:",info_wifi(w2)[1])
# # print("w3:",info_wifi(w3)[1])
# # print("w4:",info_wifi(w4)[1])


# w_prev = w1
# w_next = w2

# nw11, nw12 = get_same_aps(w1,w2)

# nw22, nw23 = get_same_aps(w2,w3)
# nw33, nw34 = get_same_aps(w3,w4)
# nw41, nw44 = get_same_aps(w1,w4)
# nw51, nw53 = get_same_aps(w1,w3)

# print("nw11:",info_wifi(nw11)[1])
# print("nw12:",info_wifi(nw12)[1])

# # print("nw22:",info_wifi(nw22)[1])
# # print("nw23:",info_wifi(nw23)[1])

# # print("nw33:",info_wifi(nw33)[1])
# # print("nw34:",info_wifi(nw34)[1])

# # print("nw41:",info_wifi(nw41)[1])
# # print("nw44:",info_wifi(nw44)[1])

# # print("nw51:",info_wifi(nw51)[1])
# # print("nw53:",info_wifi(nw53)[1])

# wifi_linear_interpolation(nw11,nw12)


In [622]:
# def adaptive_interpolation(rss_array, f1, f2):
#     n = get_num_interpolation(f1, f2)
#     interpolated_rss = []
#     for i in range(len(rss_array) - 1):
#         for j in range(n):
#             interpolated_rss.append(linear_interpolation(rss_array[i], rss_array[i + 1], n, j))
#     interpolated_rss.append(rss_array[-1]) 
#     return interpolated_rss

# def normalize(array):
#     mean = np.mean(array)
#     std_dev = np.std(array)
#     return [(val - mean) / std_dev for val in array]

# def calculate_weights(rss_array):
#     max_rss = max(rss_array)
#     return [math.exp(rss / max_rss) for rss in rss_array]

# def compute_weighted_distance(rss, rssdb, weights):
#     p = len(rss)
#     distance = 0
#     for k in range(p):
#         distance += weights[k] * (rss[k] - rssdb[k]) ** 2
#     return distance

# def mdtw_with_wls(rss, rssdb, weights):
#     n = len(rss)
#     m = len(rssdb)
#     d = np.zeros((n, m))

#     for i in range(n):
#         for j in range(m):
#             d[i][j] = compute_weighted_distance(rss[i], rssdb[j], weights)

#     D = np.zeros((n, m))
#     D[0][0] = d[0][0]

#     for i in range(1, n):
#         D[i][0] = d[i][0] + D[i - 1][0]

#     for j in range(1, m):
#         D[0][j] = d[0][j] + D[0][j - 1]

#     for i in range(1, n):
#         for j in range(1, m):
#             D[i][j] = d[i][j] + min(D[i - 1][j], D[i][j - 1], D[i - 1][j - 1])

#     return D[n - 1][m - 1]

# def weighted_least_squares(positions, distances, sigma):
#     weights = [1 / math.exp(di ** 2 / (2 * sigma ** 2)) for di in distances]
#     total_weight = sum(weights)

#     normalized_weights = [w / total_weight for w in weights]

#     estimated_x = sum(normalized_weights[i] * positions[i][0] for i in range(len(positions)))
#     estimated_y = sum(normalized_weights[i] * positions[i][1] for i in range(len(positions)))

#     return {'x': estimated_x, 'y': estimated_y}

# def match_fingerprints(user_input, database_fingerprints):
#     matched_rss = []
#     matched_dbrss = []

#     for user_network in user_input:
#         db_network = next((db_network for db_network in database_fingerprints if db_network['ssid'] == user_network['ssid']), None)
#         if db_network:
#             matched_rss.append(user_network['rss'])
#             matched_dbrss.append(db_network['rss'])

#     return {'matched_rss': matched_rss, 'matched_dbrss': matched_dbrss}

# def estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r):
#     matched = match_fingerprints(user_input, database_fingerprints)

#     filtered_rss = [filter_rss_by_threshold(rss_array, threshold) for rss_array in matched['matched_rss']]

#     interpolated_rss = [adaptive_interpolation(rss_array, f1, f2) for rss_array in filtered_rss]

#     normalized_rss = [normalize(rss) for rss in interpolated_rss]
#     normalized_dbrss = [normalize(rss) for rss in matched['matched_dbrss']]

#     weights = calculate_weights([val for sublist in interpolated_rss for val in sublist])

#     distances = [mdtw_with_wls(rss, normalized_dbrss[i], weights) for i, rss in enumerate(normalized_rss)]

#     sorted_distances = sorted(
#         [{'distance': d, 'position': (database_fingerprints[i]['x'], database_fingerprints[i]['y'])} for i, d in enumerate(distances)],
#         key=lambda x: x['distance']
#     )

#     top_r = sorted_distances[:r]

#     positions = [item['position'] for item in top_r]
#     distance_values = [item['distance'] for item in top_r]

#     estimated_position = weighted_least_squares(positions, distance_values, sigma)

#     return estimated_position

# # Example database fingerprints with positions
# database_fingerprints = [
#     {
#         'ssid': 'network1',
#         'rss': [-78, -82, -85, -88, -74],
#         'x': 5,
#         'y': 10
#     },
#     {
#         'ssid': 'network2',
#         'rss': [-61, -63, -60, -55, -52],
#         'x': 15,
#         'y': 20
#     }
# ]

# # Example user input
# user_input = [
#     {
#         'ssid': 'network1',
#         'rss': [-80, -75, -90, -60, -50]
#     },
#     {
#         'ssid': 'network2',
#         'rss': [-62, -65, -67, -70, -69]
#     }
# ]

# # Parameters for position estimation
# threshold = -70
# f1 = 0.5  # Target sampling frequency
# f2 = 0.125  # Original sampling frequency
# sigma = 1  # Configuration parameter for exponential function
# r = 2  # Number of top positions to consider

# estimated_position = estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r)

# print("Estimated Position:", estimated_position)


In [623]:
# import numpy as np
# from collections import Counter

# def euclidean_distance(point1, point2):
#     """
#     Calculate the Euclidean distance between two points.
#     """
#     return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))

# def knn(data, query, k, distance_fn=euclidean_distance):
#     """
#     k-Nearest Neighbors algorithm.
    
#     Parameters:
#     data: list of tuples (each tuple is a data point, with the last element being the label)
#     query: tuple (the data point to query)
#     k: int (the number of nearest neighbors to find)
#     distance_fn: function (a function to calculate the distance between points)
    
#     Returns:
#     The most common label among the k nearest neighbors.
#     """
#     # Calculate distances from the query point to all points in the dataset
#     distances = []
#     for index, point in enumerate(data):
#         distance = distance_fn(point[:-1], query)
#         distances.append((distance, index))
    
#     # Sort by distance and get the k nearest neighbors
#     distances.sort(key=lambda x: x[0])
#     k_nearest_indices = [index for _, index in distances[:k]]
    
#     # Get the labels of the k nearest neighbors
#     k_nearest_labels = [data[i][-1] for i in k_nearest_indices]
    
#     # Return the most common label
#     return Counter(k_nearest_labels).most_common(1)[0][0]

# # Example usage:
# data = [
#     (2.1, 3.1, 'A'),
#     (1.3, 3.3, 'A'),
#     (3.5, 2.0, 'B'),
#     (3.1, 2.2, 'B'),
#     (2.0, 1.0, 'B')
# ]

# query = (3.0, 3.0)
# k = 3
# print(knn(data, query, k))  # Output: 'A'


In [624]:


# def filter_rss_by_threshold(rss_array, threshold):
#     return [rss for rss in rss_array if rss >= threshold]

# def linear_interpolation(rss1, rss2, n, i):
#     return ((n - i) / n) * rss1 + (i / n) * rss2

# def adaptive_interpolation(rss_array, f1, f2):
#     n = math.floor(f1 / f2)
#     interpolated_rss = []
#     for i in range(len(rss_array) - 1):
#         for j in range(n):
#             interpolated_rss.append(linear_interpolation(rss_array[i], rss_array[i + 1], n, j))
#     interpolated_rss.append(rss_array[-1])  # Include the last original RSS
#     return interpolated_rss

# def normalize(array):
#     mean = np.mean(array)
#     std_dev = np.std(array)
#     return [(val - mean) / std_dev for val in array]

# def calculate_weights(rss_array):
#     max_rss = max(rss_array)
#     return [math.exp(rss / max_rss) for rss in rss_array]

# def compute_weighted_distance(rss, rssdb, weights):
#     p = len(rss)
#     distance = 0
#     for k in range(p):
#         distance += weights[k] * (rss[k] - rssdb[k]) ** 2
#     return distance
# # Generate random database fingerprints
# def generate_random_fingerprints(num_fingerprints, num_rss_values):
#     fingerprints = []
#     for _ in range(num_fingerprints):
#         ssid = f"network{_ + 1}"
#         rss = [random.randint(-90, -30) for _ in range(num_rss_values)]
#         x = random.uniform(0, 50)
#         y = random.uniform(0, 50)
#         fingerprints.append({'ssid': ssid, 'rss': rss, 'x': x, 'y': y})
#     return fingerprints

# # Generate user inputs by slightly varying the RSS values from the database
# def generate_user_inputs(database_fingerprints, num_inputs, variation):
#     user_inputs = []
#     for _ in range(num_inputs):
#         input_entry = []
#         for db_entry in random.sample(database_fingerprints, len(database_fingerprints)):
#             ssid = db_entry['ssid']
#             rss = [rss_val + random.randint(-variation, variation) for rss_val in db_entry['rss']]
#             input_entry.append({'ssid': ssid, 'rss': rss})
#         user_inputs.append(input_entry)
#     return user_inputs

# # Calculate accuracy based on estimated vs actual positions
# def calculate_accuracy(estimated_positions, actual_positions):
#     total_error = 0
#     errors = []
#     for est_pos, act_pos in zip(estimated_positions, actual_positions):
#         error = math.sqrt((est_pos['x'] - act_pos['x']) ** 2 + (est_pos['y'] - act_pos['y']) ** 2)
#         errors.append(error)
#         total_error += error
#     avg_error = total_error / len(estimated_positions)
#     return errors, avg_error

# dtw_path_from_metric(
#     s1=[[1,2,1],[1,1,2]], 
#     s2=[[1,2,3]],
#     global_constraint=None, # sakoe_chiba ,itakura
#     sakoe_chiba_radius=None, # int
#     itakura_max_slope=None, # float
#     be=None, # pytorch, numpy
#     metric=lambda x, y: np.sum((x-y)**2),
#     # list , similary score
# )

# # # Generate the data
# # num_fingerprints = 50
# # num_rss_values = 5
# # num_inputs = 10
# # variation = 5

# # database_fingerprints = generate_random_fingerprints(num_fingerprints, num_rss_values)
# # user_inputs = generate_user_inputs(database_fingerprints, num_inputs, variation)

# # # Estimate positions for user inputs
# # estimated_positions = []
# # actual_positions = [{'x': entry['x'], 'y': entry['y']} for entry in random.sample(database_fingerprints, num_inputs)]

# # threshold = -70
# # f1 = 0.5
# # f2 = 0.125
# # sigma = 1
# # r = 5

# # for user_input in user_inputs:
# #     estimated_position = estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r)
# #     estimated_positions.append(estimated_position)

# # # Calculate and print accuracy
# # errors, avg_error = calculate_accuracy(estimated_positions, actual_positions)

# # print("Individual Errors:", errors)
# # print("Average Error:", avg_error)

# # # Example database fingerprints with positions (for reproducibility)
# # print("Database Fingerprints:", database_fingerprints[:5])  # Print first 5 for brevity

# # # Example user inputs (for reproducibility)
# # print("User Inputs:", user_inputs[:3])  # Print first 3 for brevity

# # # Estimated positions
# # print("Estimated Positions:", estimated_positions)
# # def mdtw_with_wls(rss, rssdb, weights):
# #     n = len(rss)
# #     m = len(rssdb)
# #     d = np.zeros((n, m))

# #     for i in range(n):
# #         for j in range(m):
# #             d[i][j] = compute_weighted_distance(rss[i], rssdb[j], weights)

# #     D = np.zeros((n, m))
# #     D[0][0] = d[0][0]

# #     for i in range(1, n):
# #         D[i][0] = d[i][0] + D[i - 1][0]

# #     for j in range(1, m):
# #         D[0][j] = d[0][j] + D[0][j - 1]

# #     for i in range(1, n):
# #         for j in range(1, m):
# #             D[i][j] = d[i][j] + min(D[i - 1][j], D[i][j - 1], D[i - 1][j - 1])

# #     return D[n - 1][m - 1]

# # def weighted_least_squares(positions, distances, sigma):
# #     weights = [1 / math.exp(di ** 2 / (2 * sigma ** 2)) for di in distances]
# #     total_weight = sum(weights)

# #     normalized_weights = [w / total_weight for w in weights]

# #     estimated_x = sum(normalized_weights[i] * positions[i][0] for i in range(len(positions)))
# #     estimated_y = sum(normalized_weights[i] * positions[i][1] for i in range(len(positions)))

# #     return {'x': estimated_x, 'y': estimated_y}

# # def match_fingerprints(user_input, database_fingerprints):
# #     matched_rss = []
# #     matched_dbrss = []

# #     for user_network in user_input:
# #         db_network = next((db_network for db_network in database_fingerprints if db_network['ssid'] == user_network['ssid']), None)
# #         if db_network:
# #             matched_rss.append(user_network['rss'])
# #             matched_dbrss.append(db_network['rss'])

# #     return {'matched_rss': matched_rss, 'matched_dbrss': matched_dbrss}

# # def estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r):
# #     matched = match_fingerprints(user_input, database_fingerprints)

# #     filtered_rss = [filter_rss_by_threshold(rss_array, threshold) for rss_array in matched['matched_rss']]

# #     interpolated_rss = [adaptive_interpolation(rss_array, f1, f2) for rss_array in filtered_rss]

# #     normalized_rss = [normalize(rss) for rss in interpolated_rss]
# #     normalized_dbrss = [normalize(rss) for rss in matched['matched_dbrss']]

# #     weights = calculate_weights([val for sublist in interpolated_rss for val in sublist])

# #     distances = [mdtw_with_wls(rss, normalized_dbrss[i], weights) for i, rss in enumerate(normalized_rss)]

# #     sorted_distances = sorted(
# #         [{'distance': d, 'position': (database_fingerprints[i]['x'], database_fingerprints[i]['y'])} for i, d in enumerate(distances)],
# #         key=lambda x: x['distance']
# #     )

# #     top_r = sorted_distances[:r]

# #     positions = [item['position'] for item in top_r]
# #     distance_values = [item['distance'] for item in top_r]

# #     estimated_position = weighted_least_squares(positions, distance_values, sigma)

# #     return estimated_position

# # # Example database fingerprints with positions
# # database_fingerprints = [
# #     {
# #         'ssid': 'network1',
# #         'rss': [-78, -82, -85, -88, -74],
# #         'x': 5,
# #         'y': 10
# #     },
# #     {
# #         'ssid': 'network2',
# #         'rss': [-61, -63, -60, -55, -52],
# #         'x': 15,
# #         'y': 20
# #     }
# # ]

# # # Example user input
# # user_input = [
# #     {
# #         'ssid': 'network1',
# #         'rss': [-80, -75, -90, -60, -50]
# #     },
# #     {
# #         'ssid': 'network2',
# #         'rss': [-62, -65, -67, -70, -69]
# #     }
# # ]

# # # Parameters for position estimation
# # threshold = -70
# # f1 = 0.5  # Target sampling frequency
# # f2 = 0.125  # Original sampling frequency
# # sigma = 1  # Configuration parameter for exponential function
# # r = 2  # Number of top positions to consider

# # estimated_position = estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r)

# # print("Estimated Position:", estimated_position)
